# 🖼️ BLIP: Bootstrap Language-Image Pre-training

BLIP是Salesforce开发的多模态模型，支持：
- **图像描述生成** (Image Captioning)
- **视觉问答** (Visual Question Answering)
- **图文检索** (Image-Text Retrieval)

## BLIP vs CLIP 对比

| 特性 | CLIP | BLIP |
|------|------|------|
| 图像描述生成 | ❌ | ✅ |
| 视觉问答 | ❌ | ✅ |
| 图文检索 | ✅ | ✅ |
| 多目标理解 | 弱 | 强 |

## 1. 环境安装

In [ ]:
# 安装依赖
!pip install -q transformers accelerate timm
!pip install -q faiss-cpu
!pip install -q ftfy regex tqdm

In [ ]:
import os
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import requests
from io import BytesIO
import faiss

from transformers import (
    BlipProcessor, 
    BlipForConditionalGeneration,
    BlipForQuestionAnswering,
    BlipForImageTextRetrieval
)

print(f"PyTorch版本: {torch.__version__}")
print(f"CUDA可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. 加载BLIP模型

In [ ]:
class BLIPSystem:
    def __init__(self, device=None):
        if device is None:
            self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        else:
            self.device = torch.device(device)
        
        print(f"🖥️ 使用设备: {self.device}")
        
        # 加载模型
        self._load_models()
    
    def _load_models(self):
        print("📥 加载BLIP模型...")
        
        # 图像描述生成模型
        self.caption_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
        self.caption_model = BlipForConditionalGeneration.from_pretrained(
            "Salesforce/blip-image-captioning-base"
        ).to(self.device)
        
        # 视觉问答模型
        self.vqa_processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
        self.vqa_model = BlipForQuestionAnswering.from_pretrained(
            "Salesforce/blip-vqa-base"
        ).to(self.device)
        
        # 图文检索模型
        self.retrieval_processor = BlipProcessor.from_pretrained("Salesforce/blip-itm-base-coco")
        self.retrieval_model = BlipForImageTextRetrieval.from_pretrained(
            "Salesforce/blip-itm-base-coco"
        ).to(self.device)
        
        print("✅ 所有模型加载完成")
    
    def generate_caption(self, image, max_length=50):
        """
        生成图像描述
        
        Args:
            image: PIL Image或图片路径
            max_length: 最大生成长度
            
        Returns:
            生成的描述文本
        """
        if isinstance(image, str):
            image = Image.open(image).convert('RGB')
        
        inputs = self.caption_processor(image, return_tensors="pt").to(self.device)
        
        with torch.no_grad():
            output = self.caption_model.generate(
                **inputs, 
                max_length=max_length,
                num_beams=5,
                temperature=1.0,
                top_k=50,
                top_p=0.95
            )
        
        caption = self.caption_processor.decode(output[0], skip_special_tokens=True)
        return caption
    
    def answer_question(self, image, question, max_length=30):
        """
        视觉问答
        
        Args:
            image: PIL Image或图片路径
            question: 问题文本
            max_length: 最大答案长度
            
        Returns:
            答案文本
        """
        if isinstance(image, str):
            image = Image.open(image).convert('RGB')
        
        inputs = self.vqa_processor(image, question, return_tensors="pt").to(self.device)
        
        with torch.no_grad():
            output = self.vqa_model.generate(
                **inputs, 
                max_length=max_length
            )
        
        answer = self.vqa_processor.decode(output[0], skip_special_tokens=True)
        return answer
    
    def compute_similarity(self, image, text):
        """
        计算图文相似度
        
        Args:
            image: PIL Image或图片路径
            text: 文本描述
            
        Returns:
            相似度分数 (0-1)
        """
        if isinstance(image, str):
            image = Image.open(image).convert('RGB')
        
        inputs = self.retrieval_processor(image, text, return_tensors="pt").to(self.device)
        
        with torch.no_grad():
            output = self.retrieval_model(**inputs, use_itm_head=True)
            similarity = torch.softmax(output[0], dim=1)[0, 1].item()
        
        return similarity
    
    def extract_features(self, image):
        """
        提取图像特征向量
        
        Args:
            image: PIL Image或图片路径
            
        Returns:
            特征向量 (numpy array)
        """
        if isinstance(image, str):
            image = Image.open(image).convert('RGB')
        
        inputs = self.retrieval_processor(image, return_tensors="pt").to(self.device)
        
        with torch.no_grad():
            vision_outputs = self.retrieval_model.vision_model(
                pixel_values=inputs['pixel_values']
            )
            image_embeds = vision_outputs[1]
            image_embeds = self.retrieval_model.vision_proj(image_embeds)
            image_embeds = image_embeds / image_embeds.norm(dim=-1, keepdim=True)
        
        return image_embeds.cpu().numpy().astype('float32')

# 初始化系统
blip = BLIPSystem()

## 3. 图像描述生成演示

In [ ]:
def load_image_from_url(url):
    """从URL加载图片"""
    response = requests.get(url)
    return Image.open(BytesIO(response.content)).convert('RGB')

def visualize_caption(image, caption):
    """可视化图像描述结果"""
    plt.figure(figsize=(8, 6))
    plt.imshow(image)
    plt.title(caption, fontsize=14, wrap=True)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
# 测试图片URL
test_images = [
    "http://images.cocodataset.org/val2017/000000039769.jpg",  # 猫
    "http://images.cocodataset.org/val2017/000000397133.jpg",  # 人在冲浪
    "http://images.cocodataset.org/val2017/000000252219.jpg",  # 厨房场景
]

print("="*60)
print("📸 图像描述生成演示")
print("="*60)

for i, url in enumerate(test_images):
    print(f"\n--- 图片 {i+1} ---")
    
    # 加载图片
    image = load_image_from_url(url)
    
    # 生成描述
    caption = blip.generate_caption(image)
    
    print(f"生成的描述: {caption}")
    
    # 可视化
    visualize_caption(image, caption)

## 4. 视觉问答演示

In [ ]:
def visualize_vqa(image, qa_pairs):
    """可视化VQA结果"""
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # 显示图片
    axes[0].imshow(image)
    axes[0].set_title("查询图片", fontsize=14)
    axes[0].axis('off')
    
    # 显示问答
    qa_text = "\n\n".join([f"Q: {q}\nA: {a}" for q, a in qa_pairs])
    axes[1].text(0.1, 0.5, qa_text, fontsize=12, va='center',
                bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.3))
    axes[1].set_title("视觉问答结果", fontsize=14)
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# VQA测试
print("="*60)
print("❓ 视觉问答演示")
print("="*60)

# 使用第一张测试图片
image = load_image_from_url(test_images[0])

questions = [
    "What animal is in the image?",
    "How many animals are there?",
    "What color are the animals?",
    "Where are the animals?",
    "What are the animals doing?"
]

qa_pairs = []
for question in questions:
    answer = blip.answer_question(image, question)
    qa_pairs.append((question, answer))
    print(f"Q: {question}")
    print(f"A: {answer}\n")

visualize_vqa(image, qa_pairs)

## 5. 多目标场景理解演示

In [ ]:
print("="*60)
print("🎯 多目标场景理解")
print("="*60)

# 复杂场景图片
complex_scene_url = "http://images.cocodataset.org/val2017/000000439265.jpg"
image = load_image_from_url(complex_scene_url)

# 生成整体描述
caption = blip.generate_caption(image)
print(f"\n整体描述: {caption}")

# 针对性提问
questions = [
    "What objects are in the image?",
    "How many people are there?",
    "What is the background?",
    "What colors can you see?",
    "What is happening in the scene?"
]

print("\n针对性问答:")
qa_pairs = []
for question in questions:
    answer = blip.answer_question(image, question)
    qa_pairs.append((question, answer))
    print(f"  Q: {question}")
    print(f"  A: {answer}\n")

visualize_vqa(image, qa_pairs)

## 6. 图文检索功能

In [ ]:
class BLIPRetrieval:
    def __init__(self, blip_system):
        self.blip = blip_system
        self.image_index = None
        self.image_paths = []
        self.image_embeddings = []
    
    def add_images(self, image_paths, batch_size=32):
        """添加图片到索引"""
        print(f"📥 处理 {len(image_paths)} 张图片...")
        
        embeddings = []
        for i in tqdm(range(0, len(image_paths), batch_size)):
            batch_paths = image_paths[i:i+batch_size]
            batch_embeds = []
            
            for path in batch_paths:
                try:
                    embed = self.blip.extract_features(path)
                    batch_embeds.append(embed)
                    self.image_paths.append(path)
                except Exception as e:
                    print(f"处理失败 {path}: {e}")
            
            if batch_embeds:
                embeddings.extend(batch_embeds)
        
        self.image_embeddings = np.vstack(embeddings).astype('float32')
        print(f"✅ 特征提取完成，形状: {self.image_embeddings.shape}")
    
    def build_index(self):
        """构建FAISS索引"""
        d = self.image_embeddings.shape[1]
        self.image_index = faiss.IndexFlatIP(d)
        self.image_index.add(self.image_embeddings)
        print(f"✅ 索引构建完成，包含 {self.image_index.ntotal} 个向量")
    
    def search(self, query_text, k=5):
        """文本检索图片"""
        # 编码文本
        inputs = self.blip.retrieval_processor(
            text=query_text, 
            return_tensors="pt"
        ).to(self.blip.device)
        
        with torch.no_grad():
            text_outputs = self.blip.retrieval_model.text_encoder(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask']
            )
            text_embeds = text_outputs[0][:, 0, :]
            text_embeds = self.blip.retrieval_model.text_proj(text_embeds)
            text_embeds = text_embeds / text_embeds.norm(dim=-1, keepdim=True)
        
        query_embed = text_embeds.cpu().numpy().astype('float32')
        
        # 搜索
        D, I = self.image_index.search(query_embed, k)
        
        results = []
        for idx, score in zip(I[0], D[0]):
            if idx < len(self.image_paths):
                results.append((self.image_paths[idx], float(score)))
        
        return results

# 初始化检索系统
retrieval = BLIPRetrieval(blip)

In [ ]:
# 下载COCO样本图片
import os
import requests

os.makedirs("./sample_images", exist_ok=True)

sample_urls = [
    "http://images.cocodataset.org/val2017/000000039769.jpg",
    "http://images.cocodataset.org/val2017/000000397133.jpg",
    "http://images.cocodataset.org/val2017/000000252219.jpg",
    "http://images.cocodataset.org/val2017/000000439265.jpg",
    "http://images.cocodataset.org/val2017/000000001268.jpg",
    "http://images.cocodataset.org/val2017/000000002153.jpg",
    "http://images.cocodataset.org/val2017/000000002473.jpg",
    "http://images.cocodataset.org/val2017/000000002529.jpg",
]

image_paths = []
for i, url in enumerate(sample_urls):
    path = f"./sample_images/image_{i}.jpg"
    if not os.path.exists(path):
        response = requests.get(url)
        with open(path, 'wb') as f:
            f.write(response.content)
    image_paths.append(path)

print(f"✅ 下载了 {len(image_paths)} 张图片")

In [ ]:
# 构建索引
retrieval.add_images(image_paths)
retrieval.build_index()

In [ ]:
# 文本检索图片测试
print("="*60)
print("🔍 文本检索图片演示")
print("="*60)

test_queries = [
    "a cat sleeping on a bed",
    "people surfing in the ocean",
    "a kitchen with appliances",
]

for query in test_queries:
    print(f"\n查询: {query}")
    results = retrieval.search(query, k=3)
    
    for i, (path, score) in enumerate(results):
        print(f"  {i+1}. {os.path.basename(path)} (相似度: {score:.4f})")

## 7. BLIP vs CLIP 对比实验

In [ ]:
print("="*60)
print("📊 BLIP vs CLIP 功能对比")
print("="*60)

# 测试图片
test_image = load_image_from_url(test_images[0])

print("\n1️⃣ 图像描述生成:")
print(f"   BLIP: {blip.generate_caption(test_image)}")
print(f"   CLIP: ❌ 不支持")

print("\n2️⃣ 视觉问答:")
question = "What animal is this?"
print(f"   问题: {question}")
print(f"   BLIP: {blip.answer_question(test_image, question)}")
print(f"   CLIP: ❌ 不支持")

print("\n3️⃣ 图文相似度:")
text = "two cats sleeping on a pink blanket"
print(f"   文本: {text}")
print(f"   BLIP相似度: {blip.compute_similarity(test_image, text):.4f}")
print(f"   CLIP: ✅ 支持（需单独实现）")

## 8. 自定义图片测试

In [ ]:
# 上传自己的图片进行测试
from google.colab import files

print("📤 请上传一张图片...")
uploaded = files.upload()

for filename in uploaded.keys():
    image = Image.open(filename).convert('RGB')
    
    print("\n" + "="*60)
    print(f"📸 图片: {filename}")
    print("="*60)
    
    # 生成描述
    caption = blip.generate_caption(image)
    print(f"\n📝 自动描述: {caption}")
    
    # VQA
    questions = [
        "What is in this image?",
        "What colors are dominant?",
        "What is the main subject?"
    ]
    
    print("\n❓ 视觉问答:")
    for q in questions:
        a = blip.answer_question(image, q)
        print(f"   Q: {q}")
        print(f"   A: {a}\n")
    
    visualize_caption(image, caption)

## 9. 总结

### BLIP 核心优势

1. **图像描述生成** - 自动生成图片描述
2. **视觉问答** - 理解图片内容并回答问题
3. **多目标理解** - 通过VQA可以询问图片中的具体物体
4. **图文检索** - 支持跨模态检索

### 解决多目标问题的方法

| 方法 | 说明 |
|------|------|
| **VQA** | 通过提问获取具体物体信息 |
| **描述生成** | 自动列举图片中的主要物体 |
| **相似度匹配** | 检索包含特定物体的图片 |

### 适用场景

- 图像自动标注
- 智能相册管理
- 视觉内容理解
- 辅助视觉问答系统